In [1]:
import environments
import numpy as np
from agents.agents import MonteCarloPredictor

In [2]:
env = environments.make('hitstand')

In [3]:
type(env)

environments.blackjack.HitStand

In [4]:
env.action_space_description

{0: 'stand', 1: 'hit'}

In [5]:
env.reward_space_description

{-1: 'The House beats Jack',
 0: 'Draw/not terminal',
 1: 'Jack beats the House',
 1.5: 'Blackjack for Jack'}

In [6]:
env.observation_space_description

{0: "Player's total", 1: "Dealer's card value", 2: 'Player has got usable ace'}

In [7]:
env.observation_space_high

array([30, 26,  1])

In [8]:
env.observation_space_low

array([4, 2, 0])

In [9]:
agent = MonteCarloPredictor(env)

In [19]:
def run_experiment(env, agent, episodes, show):
    values = []
    for episode in range(episodes):
        if (episode+1) % show ==0:
            print('Episode {0}:'.format(episode+1))
            env.render()

        s0, reward, terminal, _ = env.reset()
        agent.evaluate_state(s0, reward, terminal)
        while not terminal:
            action = agent.follow_policy()
            state, reward, terminal, _ = env.step(action)
            agent.evaluate_state(state, reward, terminal)
    return agent.table
experiments = run_experiment(env, agent, 100_000 , 100)

Episode 100:
Jack:
	-Cards: ['4C', '6D']
	-Value : 10
Dealer:
	-Cards: ['9D']
	-Value : 9
Jack decides to: HIT
	-Cards: ['4C', '6D', '4C']
	-Value : 14
Jack decides to: HIT
	-Cards: ['4C', '6D', '4C', '10S']
	-Value : 24
Dealer:
	-Cards: ['9D']
	-Value : 9
THE HOUSE BEATS JACK
Episode 200:
Jack:
	-Cards: ['JD', '3D']
	-Value : 13
Dealer:
	-Cards: ['9S']
	-Value : 9
Jack decides to: HIT
	-Cards: ['JD', '3D', '6C']
	-Value : 19
Jack decides to: HIT
	-Cards: ['JD', '3D', '6C', '3D']
	-Value : 22
Dealer:
	-Cards: ['9S']
	-Value : 9
THE HOUSE BEATS JACK
Episode 300:
Jack:
	-Cards: ['4H', '9D']
	-Value : 13
Dealer:
	-Cards: ['10C']
	-Value : 10
Jack decides to: HIT
	-Cards: ['4H', '9D', 'AH']
	-Value : 14
Jack decides to: HIT
	-Cards: ['4H', '9D', 'AH', '2C']
	-Value : 16
Jack decides to: STAND
	-Cards: ['4H', '9D', 'AH', '2C']
	-Value : 16
Dealer:
	-Cards: ['10C', '5C', 'AH', '9H']
	-Value : 25
JACK BEATS THE HOUSE
Episode 400:
Jack:
	-Cards: ['3S', '2D']
	-Value : 5
Dealer:
	-Cards: ['2C']

In [23]:
experiments[:,:,1]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.